In [1]:
import os
import sys
# Add the project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.data_cleaning import load_and_prepare_jobs, load_normalized_user_skills, clean_text
from scripts.recomendation import JobCache
import pandas as pd
import numpy as np
from pydantic import BaseModel
import re
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import threading
import torch
import random as rd






c:\Users\Yousuf\anaconda3\envs\recomend\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Can read the GPU

In [2]:
print("torch version:", torch.__version__)
print("torch CUDA available?", torch.cuda.is_available())
print("torch CUDA version:", torch.version.cuda)

torch version: 2.6.0+cu118
torch CUDA available? True
torch CUDA version: 11.8


## Set Parameter

In [3]:
preprocessed_skill_path = r'C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Processed\skill_list.csv'
JOB_job_path    = r"C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Raw\Jobs.csv" # Where to load your job listings
TOP_K            = 7                  # How many jobs to return per request

## Load and preprocess the data

In [4]:
jobs = load_and_prepare_jobs(JOB_job_path)
jobs

,Job_title,Date-Posted,Company,Job-Type,Salary,Location,Description,job_id,title_clean,desc_clean,text
0,"Bachelor of Science, Nursing Instructor Part...",2023-04-25,Red Deer College,Full-time,NaN,"Red Deer, AB",Position Information\n\nPosition Title\n\nBach...,0,"Bachelor of Science, Nursing Instructor Part T...",Position Information Position Title Bachelor o...,"Bachelor of Science, Nursing Instructor Part T..."
1,PHD Graduate Student,2023-04-19,IDOBE Research Group,Full-time,NaN,"Edmonton, AB",The IDOBE (Intelligent Design & Operation for ...,1,PHD Graduate Student,The IDOBE (Intelligent Design & Operation for ...,PHD Graduate Student The IDOBE (Intelligent De...
2,Central Graduate Advisor,2023-04-19,University of Alberta,Full-time,"$47,210–$63,741 a year","Edmonton, AB",FGSR Grad Studies & Rsrch Admi\n\nCompetition ...,2,Central Graduate Advisor,FGSR Grad Studies & Rsrch Admi Competition No....,Central Graduate Advisor FGSR Grad Studies & R...
3,Student/Graduate Application,2023-04-20,Eagle Builders LP,Internship,NaN,"Blackfalds, AB",Eagle Builders welcomes and offers work experi...,3,Student/Graduate Application,Eagle Builders welcomes and offers work experi...,Student/Graduate Application Eagle Builders we...
4,"Advisor, Graduate Student Experience, Haskayne...",2023-04-12,University of Calgary,Full-time,NaN,"Calgary, AB",University of Calgary\nRSS Jump to Headline Ho...,4,"Advisor, Graduate Student Experience, Haskayne...",University of Calgary RSS Jump to Headline Hom...,"Advisor, Graduate Student Experience, Haskayne..."
...,...,...,...,...,...,...,...,...,...,...,...
40056,Bike Centre Volunteer (Fall 2022),NaN,Bike Centre,NaN,NaN,"Waterloo, ON",Service Description: The Bike Centre is a do-i...,40056,Bike Centre Volunteer (Fall 2022),Service Description: The Bike Centre is a do-i...,Bike Centre Volunteer (Fall 2022) Service Desc...
40057,Volunteer Program Coordinator,2023-04-05,The Food Bank of Waterloo Region,Full-time,NaN,"Kitchener, ON",Volunteer Program Coordinator\n\nAdministratio...,40057,Volunteer Program Coordinator,Volunteer Program Coordinator Administration K...,Volunteer Program Coordinator Volunteer Progra...
40058,Volunteer Coordinator,NaN,Alzheimer Society Waterloo Wellington,Part-time,NaN,"Kitchener, ON",About the job\n\nJOB SUMMARY\n\nThis position ...,40058,Volunteer Coordinator,About the job JOB SUMMARY This position manage...,Volunteer Coordinator About the job JOB SUMMAR...
40059,Summer Camp Volunteer,NaN,Little Medical School - Region of Waterloo,Part-time,NaN,"Kitchener, ON",Do you love working and interacting with child...,40059,Summer Camp Volunteer,Do you love working and interacting with child...,Summer Camp Volunteer Do you love working and ...


In [5]:
user_skills = load_normalized_user_skills(preprocessed_skill_path)
user_skills

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Yousuf\\DEPI\\Technical\\Mega Projects\\Job_Recommendation_System\\data\\Processed\\skill_list.csv'

# Recommendation

In [6]:
# Load job listings embeddings
job_emb_path_sbert = r"C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Processed\Embeddings\job_listing_embedding_norm.npy"

# Load user embeddings
user_embeddings = np.load(r'C:\Yousuf\DEPI\Technical\Mega Projects\Job_Recommendation_System\data\Processed\Embeddings\user_profile_embeddings.npy')

EMBEDDING_MODEL  = "all-MiniLM-L6-v2" # Pretrained model for embeddings

# Initialize the job cache with embeddings file path
job_cache = JobCache(JOB_job_path, EMBEDDING_MODEL,7, job_emb_path_sbert)

embeddings loaded


In [10]:
# Query job recommendations for each user embedding

i = rd.randint(0,49999)
i= 19851
user_emb = user_embeddings[i]
recommendations = job_cache.query(user_emb, user_loc=None)  # No location filter
print(f"\nRecommendations for user {i}:")
for job in recommendations:
    print(f"Job ID: {job.job_id}, Score: {job.score:.4f}, Title: {job.Job_title}")

print(f"User {i} skills")
print(user_skills[i])


Recommendations for user 19851:
Job ID: 6323, Score: 0.4494, Title: Channel Sales Executive - Video Security & Access Control (Remote Northern...
Job ID: 36415, Score: 0.4332, Title: IT System Engineer | Yearly Contract
Job ID: 25852, Score: 0.4263, Title: Business Analytics Intern (m/f/d)
Job ID: 14193, Score: 0.4251, Title: Technical Support Representative I
Job ID: 38314, Score: 0.4223, Title: IT Lead
Job ID: 6231, Score: 0.4177, Title: Senior System Administrator - Remote
Job ID: 17748, Score: 0.4162, Title: Technical Support Analyst (Hybrid/Remote)
User 19851 skills
['windows', 'configuration', 'iis', 'consulting', 'analytics', 'backup', 'healthcare', 'ssl', 'monitoring', 'upgrade']


In [8]:

user_skills = ["Python","SQL","AWS","Docker","Kubernetes","Git","CI/CD"]
model = SentenceTransformer(EMBEDDING_MODEL)
user_emb = model.encode(user_skills, normalize_embeddings=True)
recommendations = job_cache.query(user_emb, user_loc=None)  # No location filter
print(f"\nRecommendations for user :")
for job in recommendations:
    print(f"Job ID: {job.job_id}, Score: {job.score:.4f}, Title: {job.Job_title}")



ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.